In [2]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
import pandas as pd
import geopandas as gpd
from tqdm.auto import tqdm
from src.settings import *
from src.tools.osmnx_utils import get_place_dir_name
import json5 as json
tqdm.pandas()

In [4]:
with open(RAW_DATA_DIR / "featureset_transformation_default.jsonc", "r") as f:
    featureset = json.load(f)
    features = list(featureset.keys())

In [5]:
cities = pd.read_csv(RAW_DATA_DIR / "cities.csv")
cities = cities[(cities.country == "Poland") & (cities.kacper)]
cities

,city,country,continent,kacper,szymon,piotr,kamil,regions,to_fix
71,Białystok,Poland,Europe,True,False,False,False,NaN,False
72,Bydgoszcz,Poland,Europe,True,False,True,False,NaN,False
73,Częstochowa,Poland,Europe,True,False,False,False,NaN,False
77,Kraków,Poland,Europe,True,True,True,False,NaN,False
78,Lublin,Poland,Europe,True,False,True,False,NaN,False
79,Poznań,Poland,Europe,True,True,True,True,NaN,False
80,Radom,Poland,Europe,True,False,True,False,NaN,False
82,Szczecin,Poland,Europe,True,True,True,False,NaN,False
83,Trójmiasto,Poland,Europe,True,False,True,False,"['Gdańsk','Gdynia','Sopot']",False
84,Warszawa,Poland,Europe,True,True,True,True,NaN,False


In [9]:
network_type = "drive"

pbar = tqdm(cities.itertuples(), total=cities.shape[0])
edges = []
for row in pbar:
    place_name = f"{row.city},{row.country}"
    place_dir_name = get_place_dir_name(place_name)
    pbar.set_description(place_name)

    try:
        edges_city = gpd.read_file(GENERATED_DATA_DIR / place_dir_name / f"graph_{network_type}.gpkg", layer="edges")
        cols_to_add = set(features) - set(edges_city.columns)
        edges_city[list(cols_to_add)] = None
        edges_city["city"] = row.city
        edges_city["country"] = row.country
        edges_city["continent"] = row.continent
        edges.append(edges_city[features])
    except Exception as e:
        print("\nFailed", place_name, e)
    

edges = pd.concat(edges, ignore_index=True)
del edges_city

Łódź,Poland: 100%|██████████| 13/13 [00:06<00:00,  1.93it/s]        


In [8]:
edges.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 126921 entries, 0 to 126920
Data columns (total 9 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   oneway    126921 non-null  bool  
 1   lanes     126921 non-null  object
 2   highway   126921 non-null  object
 3   maxspeed  126921 non-null  object
 4   bridge    126921 non-null  object
 5   access    126921 non-null  object
 6   junction  126921 non-null  object
 7   width     126921 non-null  object
 8   tunnel    126921 non-null  object
dtypes: bool(1), object(8)
memory usage: 7.9+ MB


In [10]:
from itertools import chain

for f in features:
    feature_values_dirty = list(edges[f].value_counts().keys())
    feature_values_dirty = list(map(lambda x: eval(x) if "[" in str(x) else [str(x)], feature_values_dirty))
    feature_values_dirty = list(set(map(lambda x: str(x), chain(*feature_values_dirty))))
    feature_values_dirty.sort()
    print(f+":", feature_values_dirty, "\n")

oneway: ['False', 'True'] 

lanes: ['', '1', '2', '20', '3', '4', '5', '6', '7'] 

highway: ['bus_stop', 'living_street', 'motorway', 'motorway_link', 'primary', 'primary_link', 'residential', 'secondary', 'secondary_link', 'tertiary', 'tertiary_link', 'trunk', 'trunk_link', 'unclassified'] 

maxspeed: ['', '10', '100', '110', '120', '140', '15', '2', '20', '25', '30', '40', '5', '50', '60', '70', '80', '90'] 

bridge: ['', 'low_water_crossing', 'movable', 'viaduct', 'yes'] 

access: ['', 'construction', 'customers', 'delivery', 'designated', 'destination', 'forestry', 'no', 'permissive', 'permit', 'psv', 'residents', 'restricted', 'yes'] 

junction: ['', 'circular', 'intersection', 'roundabout', 'yes'] 

width: ['', '0', '1', '1.5', '1.7', '1.88', '10', '10.5', '11', '12', '12.5', '13', '13.5', '14', '14.5', '15', '15.5', '16', '16.5', '17.5', '18', '18.5', '2', '2.06', '2.22', '2.49', '2.5', '3', '3.07', '3.25', '3.39', '3.5', '3.75', '3.9', '4', '4.5', '4.50', '4.75', '4m', '5', '5.